# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-06-21&until-occurred-date=2021-06-28


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

10567 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-21&until-occurred-date=2021-06-28
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=c7b2df1c-b41f-4850-848b-0130da7badf4&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-21&until-occurred-date=2021-06-28
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=0bcbbbc9-586a-4265-bdc6-ac647661a456&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-21&until-occurred-date=2021-06-28
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=2d7209b8-315c-4278-80c3-e8e58307ba9d&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-06-21.csv
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-06-14.csv
failed to load preprint_tweets_2021-06-07.csv
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-24.csv
failed to load preprint_tweets_2021-05-31.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.01.11.425914...
REST API query complete  200
REST API query started for 10.1101/2021.06.09.447686...
REST API query complete  200
REST API query started for 10.1101/2020.12.21.423721...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.05.22.21257658...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2021.04.27.21256187...
REST API query complete  200
REST API query started for 10.1101/2021.04.15.21252192...
REST API query complete  200
REST API query started for 10.1101/2021.06.20.449152...
REST API query complete  200
REST API query started for 10.1101/2021.06.23.449568...
REST API query complete  200
REST API query started for 10.1101/2021.06.16.21259052.

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.06.20.449152,199,bioRxiv,Neuroscience,False,Large-scale neural recordings with single-cell...,"[{'name': 'Angelique C. Paulk'}, {'name': 'Yoa...",<p>Recent advances in multi-electrode array te...,2021-06-21
1,10.1101/2021.06.23.449568,196,bioRxiv,Molecular Biology,True,Increased lung cell entry of B.1.617.2 and eva...,"[{'name': 'Prerna Arora'}, {'name': 'Amy Kempf...","<p>The delta variant of SARS-CoV-2, B.1.617.2,...",2021-06-23
2,10.1101/2021.06.24.449497,38,bioRxiv,Zoology,False,A novel flight style allowing the smallest fea...,"[{'name': 'Sergey E. Farisenkov'}, {'name': 'D...",<p>Flight speed generally correlates positivel...,2021-06-25
3,10.1101/2021.06.22.449254,34,bioRxiv,Cell Biology,False,RNA at the surface of phase-separated condensa...,"[{'name': 'Audrey Cochard'}, {'name': 'Marina ...","<p>Membrane-less organelles, by localizing and...",2021-06-22
4,10.1101/2021.06.21.449205,26,bioRxiv,Evolutionary Biology,True,One viral sequence for each host? – The neglec...,"[{'name': 'Yongsen Ruan'}, {'name': 'Mei Hou'}...",<p>The standard practice of presenting one vir...,2021-06-21
5,10.1101/2021.06.15.21258669,18,medRxiv,Infectious Diseases (except HIV/AIDS),True,COMPARATIVE IMMUNOGENICITY OF BNT162b2 mRNA VA...,"[{'name': 'Mina Psichogiou'}, {'name': 'Andrea...",<p>The mRNA vaccine BNT162b2 has proven highly...,2021-06-21
6,10.1101/2021.06.24.449808,18,bioRxiv,Plant Biology,False,Methods for in situ quantitative root biology,"[{'name': 'Taras Pasternak'}, {'name': 'Jose M...","<p>When dealing with plant roots, a multi-scal...",2021-06-25
7,10.1101/2021.06.21.449174,17,bioRxiv,Molecular Biology,False,Protective role of the Atg8 homologue Gabarapl...,"[{'name': 'Kimberley M. Mellor'}, {'name': 'Up...",<p>Diabetic heart disease is highly prevalent ...,2021-06-22
8,10.1101/2021.06.25.449942,15,bioRxiv,Plant Biology,False,Intraspecific diversity observed in the wild t...,"[{'name': 'Parvinderdeep S. Kahlon'}, {'name':...",<p>Natural plant populations are highly polymo...,2021-06-26
9,10.1101/2021.06.21.449293,14,bioRxiv,Plant Biology,False,An artificial host system enables the obligate...,"[{'name': 'Vivian Bernal-Galeano'}, {'name': '...",<p><italic>Cuscuta campestris</italic> is an o...,2021-06-21


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

35 preprints (including 9 covering SARS-CoV-2, 27 from bioRxiv and 8 from medRxiv) published in the last 7 days before 2021-06-28 had been tweeted at least 5 times (maximum 199).